In [1]:
import cv2
import numpy as np 


# Object Detection Using Contours

In [9]:
def nothing(x):
    pass

cv2.namedWindow('hancy')
cv2.createTrackbar("th","hancy",0,255,nothing)

cv2.createTrackbar("lb","hancy",0,255,nothing)
cv2.createTrackbar("lg","hancy",0,255,nothing)
cv2.createTrackbar("lr","hancy",0,255,nothing)

cv2.createTrackbar("hb","hancy",255,255,nothing)
cv2.createTrackbar("hg","hancy",255,255,nothing)
cv2.createTrackbar("hr","hancy",255,255,nothing)

cap = cv2.VideoCapture(0)
while cap.isOpened():
    r,frame = cap.read()
    if r == True:
        frame = cv2.flip(frame,1)
        frame = cv2.resize(frame,(400,400))
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        thr = cv2.getTrackbarPos('th','hancy')
        Lb = cv2.getTrackbarPos("lb","hancy")
        Lg = cv2.getTrackbarPos("lg","hancy")
        Lr = cv2.getTrackbarPos("lr","hancy")
    
        Ub = cv2.getTrackbarPos("hb","hancy")
        Ug = cv2.getTrackbarPos("hg","hancy")
        Ur = cv2.getTrackbarPos("hr","hancy")

        lower = np.array([Lb,Lg,Lr])
        upper = np.array([Ub,Ug,Ur])
        masks = cv2.inRange(hsv,lower,upper)
        res = cv2.bitwise_and(frame,frame,mask=masks)
        fr = cv2.bitwise_not(res)

        _,thi = cv2.threshold(masks,thr,255,cv2.THRESH_BINARY)
        cnt,hr = cv2.findContours(thi, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

        cv2.drawContours(frame,cnt,-1,(255,0,0),2)
        cv2.imshow('thr',thi)
        cv2.imshow("res",res)
        cv2.imshow("mask",masks)
        cv2.imshow("cv",frame)
        if cv2.waitKey(25) & 0xff == ord("p"):
                break
    else:
        break

cap.release()  
cv2.destroyAllWindows()

# Image Background Using Histogram

In [7]:
img = cv2.imread("Images/thor.jpg")
img = cv2.resize(img,(600,500))
hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)

img1 = cv2.imread("Images/cropthor.jpg")
hsv1 = cv2.cvtColor(img1,cv2.COLOR_BGR2HSV)

hist = cv2.calcHist([hsv1],[0,1],None,[180,256],[0,180,0,256])
mask = cv2.calcBackProject([hsv],[0,1],hist,[0,180,0,256],1)

# res = cv2.bitwise_or(img,img,mask = mask)
# this above commented one remove the thor but we want to remove background so let's make more filter
ker = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
mask = cv2.filter2D(mask,-1,ker)
_,thr = cv2.threshold(mask,230,255,cv2.THRESH_BINARY)
mask = cv2.merge((mask,mask,mask))

res = cv2.bitwise_or(img,mask)

cv2.imshow('res',res)
cv2.imshow("opencv",img)
cv2.waitKey()
cv2.destroyAllWindows()

# Hough Transformation Lines

In [6]:
img = cv2.imread("Images/sudoku.png")
gry = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

edg = cv2.Canny(gry,20,250)
lines = cv2.HoughLines(edg,1,np.pi/180,200)

for r,th in lines[0]:
    a = np.cos(th)
    b = np.sin(th)

    x0 = a*r
    y0 = b*r 
    
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))

    cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)
    
cv2.imshow("opencv",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
img = cv2.imread("Images/sudoku.png")
gry = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

edg = cv2.Canny(gry,20,250)

l = cv2.HoughLinesP(edg,1,np.pi/180,200,minLineLength=180, maxLineGap = 100)

for i in l:
    x1,y1,x2,y2 = i[0]
    cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)

cv2.imshow("opencv",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Template Matching Using Opencv

In [48]:
img = cv2.imread("Images/heros1.jpg")
gry = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

temp = cv2.imread("Images/heroine.jpg")
gry1 = cv2.cvtColor(temp,cv2.COLOR_BGR2GRAY)

w,h = gry1.shape[::-1]
res = cv2.matchTemplate(gry,gry1,cv2.TM_CCOEFF_NORMED)
thrr = 0.94

l = np.where(res>=thrr)
for i in zip(*l[::-1]):
    cv2.rectangle(img,i,(i[0]+w,i[1]+h),(0,255,0),2)

img = cv2.resize(img,(400,300))

# cv2.imshow("res",res)
cv2.imshow("opencv",img)
cv2.imshow("opencv1",temp)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
#(96, 94)
w,h = gry1.shape[::-1]
w,h

(94, 96)

# Hough Circle detection 

In [78]:
img = cv2.imread("Images/balls.webp")
img = cv2.resize(img,(600,600))

gr = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
gr = cv2.medianBlur(gr,7)

c = cv2.HoughCircles(gr,cv2.HOUGH_GRADIENT,1,20,param1=100,param2=50,minRadius=0,maxRadius=0)
data = np.uint16(np.around(c))

for (x,y,r) in data[0,:]:
    cv2.circle(img,(x,y),r,(0,0,255),4)

cv2.imshow("opencv1",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Grabcut Algorithm for Background change

In [81]:
img = cv2.imread("Images/carbmw.jpg")

mask = np.zeros(img.shape[:2],np.uint8)

bgmask = np.zeros((1,65),np.float64)*255
fgmask = np.zeros((1,65),np.float64)*255

#x1,y1,x2,y2
r = [184,111,538,349]
cv2.grabCut(img,mask,r,bgmask,fgmask,10,cv2.GC_INIT_WITH_RECT)

mask2 = np.where((mask ==2)|(mask == 0),0,1).astype("uint8")

img = img*mask2[:,:,np.newaxis]

cv2.imshow("open",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Video Background Removal using Algorithm

In [90]:
cap = cv2.VideoCapture('Images/oceanhideko.mp4')

algo1 = cv2.createBackgroundSubtractorKNN(detectShadows=True)
algo2 = cv2.createBackgroundSubtractorMOG2(detectShadows=True)


while cap.isOpened():
    r,frame = cap.read() 
    if r == True:
        frame = cv2.resize(frame,(600,400))
        r1 = algo1.apply(frame)
        r2 = algo2.apply(frame)
        cv2.imshow("algo1",r1)
        cv2.imshow("algo2",r2)
        cv2.imshow("algokeinai",frame)
        
        if cv2.waitKey(40) & 0xff == ord('p'): 
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
#(656, 368, 3)